This is not a difficult exercise, but you need to revise the following technique:
- clean content of a column
- filter column with condition
- create dummy columns from a column which has value of a list
- group by by a column 

In [18]:
import pandas as pd

# Sample DataFrame
data = {'state_id': [1, 1, 2, 2, 3],
        'school_id': [101, 102, 201, 202, 301],
        'subjects': ['engli%sh math chemi%stry', 'math chem%istry', 'engli%sh', 'math en%glish', 'chemistry']}
df = pd.DataFrame(data)


In [19]:
df

,state_id,school_id,subjects
0,1,101,engli%sh math chemi%stry
1,1,102,math chem%istry
2,2,201,engli%sh
3,2,202,math en%glish
4,3,301,chemistry


### 1. Clean the dirty_column to contain only alphanumeric characters

In [20]:
# Clean the dirty_column to contain only alphanumeric characters
df['subjects'] = df['subjects'].str.replace(r'[^a-zA-Z0-9 ]', '', regex=True)

In [21]:
df

,state_id,school_id,subjects
0,1,101,english math chemistry
1,1,102,math chemistry
2,2,201,english
3,2,202,math english
4,3,301,chemistry


### 2. Filter the school with number of subjects > 1

In [23]:
# Define a function to count the number of words in a string
word_count = lambda x: len(x.split())
# Apply the function to the text_column and create a boolean mask
condition = df['subjects'].apply(word_count) > 1
df = df[condition]
df

,state_id,school_id,subjects
0,1,101,english math chemistry
1,1,102,math chemistry
3,2,202,math english


### 3.Split the subjects column into different columns such as each column represent for one subject of shool

In [24]:
# Split the subjects column into separate rows
df['subjects'] = df['subjects'].str.strip().str.split(" ")

/tmp/ipykernel_3471/2339059929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjects'] = df['subjects'].str.strip().str.split(" ")


In [25]:
df

,state_id,school_id,subjects
0,1,101,"[english, math, chemistry]"
1,1,102,"[math, chemistry]"
3,2,202,"[math, english]"


In [26]:
# Convert the list of subjects into dummy columns
# Solution 1. Create dummy columns for each subject
for subject in set(subject for sublist in df['subjects'] for subject in sublist):
    df[subject] = df['subjects'].apply(lambda x: 1 if subject in x else 0)


/tmp/ipykernel_3471/1335810729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[subject] = df['subjects'].apply(lambda x: 1 if subject in x else 0)
/tmp/ipykernel_3471/1335810729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[subject] = df['subjects'].apply(lambda x: 1 if subject in x else 0)
/tmp/ipykernel_3471/1335810729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [34]:
# # Solution 2
# # Convert the list of subjects into dummy columns
# dummy_subjects = pd.get_dummies(df['subjects'].apply(pd.Series).stack()).sum(level=0)
# # Combine the original DataFrame with the dummy columns
# df = pd.concat([df, dummy_subjects], axis=1)

In [27]:
df

,state_id,school_id,subjects,english,math,chemistry
0,1,101,"[english, math, chemistry]",1,1,1
1,1,102,"[math, chemistry]",0,1,1
3,2,202,"[math, english]",1,1,0


In [28]:
# Drop the original 'subjects' column
df.drop('subjects', axis=1, inplace=True)

df

/tmp/ipykernel_3471/1610880462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('subjects', axis=1, inplace=True)


,state_id,school_id,english,math,chemistry
0,1,101,1,1,1
1,1,102,0,1,1
3,2,202,1,1,0


### 4. Group by state_id to see the subject of each state

In [31]:
df_state = df.drop('school_id', axis=1)

In [32]:
sum_by_state = df_state.groupby('state_id').sum().reset_index()

In [33]:
sum_by_state

,state_id,english,math,chemistry
0,1,1,2,2
1,2,1,1,0
